In [42]:
import pandas as pd
import numpy as np
#from datetime import datetime

In [3]:
# read in relevant cols, convert timestamp col to datetime
cols = pd.read_csv("phq_scores_niaaa_20230715.csv", nrows=1).columns
df = pd.read_csv("phq_scores_niaaa_20230715.csv", usecols=cols[1:])
#df['timestamp'] = df['timestamp'].apply(lambda x: datetime.strptime(x.split('.')[0], '%Y-%m-%dT%H:%M:%S'))
#df['timestamp'] = df['timestamp'].apply(lambda x: np.datetime64(x))
df['timestamp'] = df['timestamp'].apply(lambda x: pd.to_datetime(x))
df

,user_id,subject_id,timestamp,survey_name,phq_total
0,us-east-1:00c110a8-18fa-4ab9-ba23-474b1f809359,MM293AUD,2023-01-30 18:38:57.101000+01:00,phq8,2
1,us-east-1:00f63ac3-c54e-4e76-897b-e1dc68cb72d9,CM236AUD,2022-12-21 10:04:08.550000-08:00,phq8,0
2,us-east-1:00f63ac3-c54e-4e76-897b-e1dc68cb72d9,CM236AUD,2023-01-05 13:44:03.620000-08:00,phq8,1
3,us-east-1:00f63ac3-c54e-4e76-897b-e1dc68cb72d9,CM236AUD,2023-01-21 08:03:52.336000+01:00,phq8,2
4,us-east-1:00f63ac3-c54e-4e76-897b-e1dc68cb72d9,CM236AUD,2023-02-01 16:39:49.721000+01:00,phq8,0
...,...,...,...,...,...
731,us-east-1:f8452db9-2ae8-4b94-8dd4-ff8ea21d3235,MF035AUD,2022-03-20 22:08:50.675000-06:00,phq8,6
732,us-east-1:fc5296f1-c47a-42c6-93ef-13e9d3a8af5c,CM059AUD,2021-12-30 22:01:00.356000-07:00,phq8,5
733,us-east-1:fc5296f1-c47a-42c6-93ef-13e9d3a8af5c,CM059AUD,2022-01-13 14:48:58.558000-07:00,phq8,2
734,us-east-1:fc5296f1-c47a-42c6-93ef-13e9d3a8af5c,CM059AUD,2022-01-27 08:51:52.652000-07:00,phq8,3


In [5]:
df.iloc[0,2].date()

datetime.date(2023, 1, 30)

In [7]:
# read in basline date reference
df_ref = pd.read_csv("HealthRhythmsAUDCTSI-BaselineDates_DATA_2023-07-27_1631.csv")
#df_ref.dropna(inplace=True)
df_ref = df_ref.loc[df_ref['record_id'] != 'PM142AUD']
#df_ref['visit_date'] = df_ref['visit_date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
#df_ref['visit_date'] = df_ref['visit_date'].apply(lambda x: np.datetime64(x))
df_ref['visit_date'] = df_ref['visit_date'].apply(lambda x: pd.to_datetime(x))
df_ref

,record_id,redcap_event_name,visit_date,eligible_status
0,CF017AUD,baseline_arm_3,2021-11-19,1
1,CF021AUD,baseline_arm_3,2021-11-22,1
2,CF026AUD,baseline_arm_3,2021-11-30,1
3,CF037AUD,baseline_arm_3,2021-12-14,1
4,CF041AUD,baseline_arm_3,2021-12-07,1
...,...,...,...,...
136,MM293AUD,baseline_arm_3,2023-01-29,1
137,MM312AUD,baseline_arm_3,2023-04-23,1
138,MM315AUD,baseline_arm_3,2023-04-29,1
139,MM318AUD,baseline_arm_3,2023-05-02,1


In [8]:
df_ref.iloc[0,2].date()

datetime.date(2021, 11, 19)

In [43]:
def get_study_week(row):
    baseline_date = pd.to_datetime(df_ref.loc[df_ref['record_id']==row['subject_id'],['visit_date']].values[0][0]).date()
    date_of_interest = row['timestamp'].date()
    td = date_of_interest-baseline_date
    return round(td.days/7,0)

#get_study_week(df.iloc[0])

In [44]:
for i, row in df.iterrows():
    try:
        df.loc[df.index[i],"study_week"] = get_study_week(row)
    except:
        df.loc[df.index[i],"study_week"] = np.nan

In [47]:
df['study_week'].value_counts()

0.0     108
2.0      99
4.0      87
6.0      84
8.0      74
12.0     73
10.0     67
14.0     46
5.0       9
1.0       8
11.0      8
13.0      6
3.0       5
9.0       5
7.0       4
15.0      3
16.0      2
17.0      1
18.0      1
Name: study_week, dtype: int64